In [3]:

pip install --upgrade tensorflow -q


Note: you may need to restart the kernel to use updated packages.


## IMPORT PACKAGES

In [4]:
import os
import cv2
import random
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")

import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Flatten, Dense, Lambda, Dropout, Activation)


from tensorflow.keras.layers import (Conv2D, MaxPooling2D)
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [1]:
pip install keras -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
import keras

In [6]:
STYLES_CSV_PATH = "/home/jovyan/efs/users/Shreya_Sivakumar/TASK_1/data/styles.csv"
IMAGES_PATH = "/home/jovyan/efs/users/Shreya_Sivakumar/TASK_1/data/images/"

In [7]:
df = pd.read_csv(STYLES_CSV_PATH, error_bad_lines = False, warn_bad_lines=False)
print(f"Total Rows: {df.shape[0]}\nTotal Columns: {df.shape[1]}")
df.head()

Total Rows: 44424
Total Columns: 10


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [8]:
print('NaN Count:')
df.isna().sum()

NaN Count:


id                      0
gender                  0
masterCategory          0
subCategory             0
articleType             0
baseColour             15
season                 21
year                    1
usage                 317
productDisplayName      7
dtype: int64

In [9]:
df = df.dropna()
df = df.drop(['gender', 'articleType','season','usage','year', 'productDisplayName', 'masterCategory', 'subCategory'], axis=1)
df = df.sample(10000)
df = df[df['id'].isin([int(i.split('.')[0]) for i in os.listdir(IMAGES_PATH)])]

In [36]:
df.head()

,baseColour
28125,Purple
10000,Grey
23599,Brown
37684,Navy Blue
42112,Brown


In [39]:
unique_colors = df['baseColour'].unique()

# Print the unique color values
print("Unique Colour Values:")
for color in unique_colors:
  print(color)

Unique Colour Values:
Purple
Grey
Brown
Navy Blue
Black
Silver
Green
Orange
White
Blue
Other
Multi
Beige
Steel
Yellow
Olive
Pink
Gold
Red
Maroon
Cream


# Pre-processing

In [10]:
df['id'] = df['id'].apply(lambda x: IMAGES_PATH+str(x) +'.jpg')
image_ids = df.pop('id')

In [11]:
def bin_values(x):
    x_vc = df[x].value_counts()
    x_other = x_vc[x_vc<50].index
    df.loc[df[x].isin(x_other),x] = 'Other'
    
for col in df.columns:
    bin_values(col)

Load Images

In [12]:
from keras.applications.resnet import preprocess_input

IMAGE_DIMS = (60, 60, 3)

def load_image(imagePath):
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = preprocess_input(image)
    return image

image_data = []
for img_path in tqdm(image_ids):
    image_data.append(load_image(img_path))
    
image_data = np.array(image_data, dtype="float")

100%|██████████| 10000/10000 [04:12<00:00, 39.58it/s]


Load other features

In [13]:

baseColourLB = LabelBinarizer()


baseColourLabels = baseColourLB.fit_transform(np.array(df['baseColour'].values))

split = train_test_split(image_data,
                         baseColourLabels, 
                         test_size=0.2, random_state=42)

(trainX, testX,
 trainBaseColourY, testBaseColourY) = split

# LOAD MODEL

In [14]:
from tensorflow.keras.applications import ResNet50

In [15]:
def make_branch(res_input, n_out, act_type, name):
    z = Dense(512, activation="relu")(res_input)
    z = Dense(256, activation='relu')(z)
    z = Dense(128, activation='relu')(z)
#     z = BatchNormalization()(z)
#     z = Dropout(0.5)(z)
    z = Dense(n_out)(z)
    z = Activation(act_type, name=name+'_output')(z)
    return z

In [17]:
def build_model(width, height):

    # -------------------------
    res50 = ResNet50(weights='imagenet', include_top=False, input_shape=IMAGE_DIMS)
    res50.trainable=False
    inputs = Input(shape=IMAGE_DIMS)
    x = res50(inputs, training=False)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    # -------------------------

    color_branch = make_branch(x, len(baseColourLB.classes_), 'softmax', 'color')


    model = Model(inputs=inputs,
                outputs=[ color_branch])
    return model

In [19]:
from keras.optimizers import Adam

EPOCHS = 25
INIT_LR = 1e-5
BS = 32

opt = Adam(learning_rate=INIT_LR)
model.compile(optimizer=opt, loss=losses, metrics=["accuracy"])

In [23]:
H = model.fit(trainX,
    {"color_output": trainBaseColourY},
    validation_data=(testX, 
    {"color_output": testBaseColourY}),
    epochs=EPOCHS,
    batch_size=BS,
    verbose=1)

Epoch 1/25
250/250 [==============================] - 88s 342ms/step - loss: 2.3728 - accuracy: 0.3316 - val_loss: 1.9999 - val_accuracy: 0.4400
Epoch 2/25
250/250 [==============================] - 90s 359ms/step - loss: 1.7779 - accuracy: 0.4959 - val_loss: 1.7944 - val_accuracy: 0.4905
Epoch 3/25
250/250 [==============================] - 85s 340ms/step - loss: 1.4915 - accuracy: 0.5739 - val_loss: 1.6870 - val_accuracy: 0.5085
Epoch 4/25
250/250 [==============================] - 85s 340ms/step - loss: 1.2870 - accuracy: 0.6276 - val_loss: 1.6185 - val_accuracy: 0.5280
Epoch 5/25
250/250 [==============================] - 85s 340ms/step - loss: 1.1169 - accuracy: 0.6811 - val_loss: 1.5916 - val_accuracy: 0.5305
Epoch 6/25
250/250 [==============================] - 85s 340ms/step - loss: 0.9779 - accuracy: 0.7287 - val_loss: 1.5549 - val_accuracy: 0.5425
Epoch 7/25
250/250 [==============================] - 86s 343ms/step - loss: 0.8594 - accuracy: 0.7601 - val_loss: 1.5416 - val_ac

In [26]:
model.save('/home/jovyan/efs/users/Shreya_Sivakumar/TASK_1/models/model.h5')


In [29]:
res = model.evaluate(testX, {"color_output": testBaseColourY}, batch_size=32, verbose=0)
print('Loss:', res[0])
# Assuming you have one metric (accuracy), it will be at index 1
print('Accuracy:', round(res[1] * 100, 2), '%')


Loss: 1.7856084108352661
Accuracy: 55.05 %


In [33]:
idx=3

colorProba= model.predict(np.expand_dims(testX[idx], axis=0))


colorIdx = colorProba[0].argmax()
colorLabel = baseColourLB.classes_[colorIdx]


colorText = "Color: {} ({:.2f}%)".format(colorLabel, colorProba[0][colorIdx] * 100)



print(colorText, '-----',baseColourLB.classes_[testBaseColourY[idx].argmax()])


1/1 [==============================] - 0s 31ms/step
Color: Grey (86.88%) ----- White


## Testing with custom image

In [42]:
# Import necessary libraries
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load the model
model_path = '/home/jovyan/efs/users/Shreya_Sivakumar/TASK_1/models/model.h5' # or 'my_model' for the SavedModel format
model = load_model(model_path)

# Load and preprocess an image
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)

# Assuming you have an image 'test_image.jpg'
img_path = '/home/jovyan/efs/users/Shreya_Sivakumar/TASK_1/data/images2.jpeg'
preprocessed_img = load_and_preprocess_image(img_path, target_size=(60, 60))  # Make sure to pass the correct target size

# Predict the color
predictions = model.predict(preprocessed_img)

# Assuming your model outputs probabilities for each class in the 'color' branch
# and you have a list of class names corresponding to the model's outputs
color_classes =[
    "Purple", "Grey", "Brown", "Navy Blue", "Black", "Silver",
    "Green", "Orange", "White", "Blue", "Other", "Multi", "Beige",
    "Steel", "Yellow", "Olive", "Pink", "Gold", "Red", "Maroon", "Cream",
    "Charcoal", "Skin", "Grey Melange", "Tan", "Khaki", "Turquoise Blue",
    "Teal", "Mustard", "Off White", "Burgundy", "Magenta", "Lavender",
    "Peach", "Copper", "Rust", "Mauve", "Nude", "Metallic", "Sea Green",
    "Rose", "Fluorescent Green", "Bronze", "Taupe", "Coffee Brown",
    "Lime Green", "Mushroom Brown"
]
  # Example class names
predicted_color = color_classes[np.argmax(predictions)]

print(f'Predicted color: {predicted_color}')


1/1 [==============================] - 1s 609ms/step
Predicted color: Maroon
